# Get data about teams and players

In [1]:
import requests
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_columns', None)

## Get basic player and team details

In [3]:
response = requests.get('https://en.fantasy.allsvenskan.se/api/bootstrap-static/')
data = response.json();
player_data = pd.DataFrame(data['elements'])
player_data.to_csv('data/player_details_raw.csv', index=False)
player_data.sample(10)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,own_goals,attacking_bonus,defending_bonus,crosses,key_passes,big_chances_created,clearances_blocks_interceptions,recoveries
173,100.0,100.0,505995,0,0,-1,1,0,2,2.4,2.4,2,Tobias,2.4,141,False,,2021-04-24T06:42:57.115757Z,44,505995.jpg,2.9,Carlsson,0.5,False,None,a,6,1959,20,154,4,68,0,0.5,4.5,Carlsson,585,0,0,2,9,0,0,1,0,0,1,0,5,1,2,0,36,26
11,0.0,0.0,89198,0,0,0,0,0,3,0.0,0.0,0,Arnór Ingvi,0.0,12,False,Lämnat för New England Revolution,2021-03-16T17:02:05.300231Z,80,89198.jpg,0.0,Traustason,0.0,False,None,u,1,993,0,0,0,4,0,0.0,0.0,Traustason,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
345,NaN,NaN,242695,0,0,-3,3,1,4,0.8,0.8,0,Francis,0.8,296,False,,None,47,242695.jpg,2.9,Jno-Baptiste,1.6,False,None,a,11,5576,29,678,6,695,10,0.2,6.2,Jno-Baptiste,486,2,1,2,7,0,0,1,0,0,0,1,0,1,7,0,11,18
236,NaN,NaN,484393,0,0,-2,2,0,2,0.0,0.0,0,Niclas,0.0,200,False,,None,38,484393.jpg,1.0,Bergmark,2.4,False,None,a,8,2596,2,450,16,578,5,0.0,0.5,Bergmark,23,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,1
290,100.0,100.0,207276,0,0,-1,1,1,2,2.0,2.0,3,Marco,2.0,247,False,,2021-07-06T07:33:07.510374Z,54,207276.jpg,3.2,Lund,0.7,False,None,a,9,483,45,305,0,375,0,0.4,8.3,Lund,1137,0,0,2,14,0,0,4,0,0,0,0,17,0,2,0,89,86
229,100.0,100.0,479393,0,0,3,-3,1,1,5.4,5.4,8,Jacob,5.4,436,False,,2021-04-15T12:31:27.637451Z,48,479393.jpg,5.9,Widell Zetterström,8.1,False,None,a,7,1406,53,3327,50,270,12,1.1,11.0,Zetterström,810,0,0,6,6,0,0,0,0,15,0,0,12,0,1,0,13,89
489,100.0,100.0,249042,0,0,-4,4,0,4,1.8,1.8,1,Sargon,1.8,417,False,,2021-07-16T18:06:15.884978Z,56,249042.jpg,2.2,Abraham,4.2,False,None,a,16,2961,28,351,0,2667,5,0.3,5.0,Abraham,744,0,1,1,12,0,0,0,0,0,0,3,1,1,8,3,6,21
334,NaN,NaN,226930,0,0,0,0,0,3,1.2,1.2,1,Henrik,1.2,285,False,,None,45,226930.jpg,2.6,Bellman,0.8,False,None,a,11,5576,36,196,0,244,4,0.3,8.0,Bellman,1013,0,2,2,23,0,0,2,0,0,0,1,4,0,5,1,12,57
197,100.0,100.0,157740,0,0,-2,2,0,3,0.4,0.4,1,Samuel,0.4,463,False,,2021-07-29T10:59:37.747853Z,68,157740.jpg,1.0,Gustafson,0.0,False,None,a,6,1959,3,38,0,21,0,0.1,0.4,Gustafson,113,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,6
415,NaN,NaN,156601,0,0,-4,4,0,3,0.8,0.8,3,Elias,0.8,184,False,,None,66,156601.jpg,1.8,Andersson,2.5,False,None,a,14,2360,7,17,0,1145,1,0.1,1.1,Andersson,158,0,0,1,2,0,0,0,0,0,0,0,1,3,2,0,2,12


In [4]:
teams_data = pd.DataFrame(data['teams'])
teams_data.to_csv('data/teams_raw.csv', index=False)
teams_data.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win
0,993,0,None,1,0,Malmö FF,0,0,0,MFF,None,None,False,0
1,1987,0,None,2,0,Hammarby,0,0,0,HAM,None,None,False,0
2,3056,0,None,3,0,IK Sirius,0,0,0,IKS,None,None,False,0
3,244,0,None,4,0,IFK Göteborg,0,0,0,GBG,None,None,False,0
4,375,0,None,5,0,IF Elfsborg,0,0,0,IFE,None,None,False,0


## Get all players game history

In [5]:
for index, player in player_data.iterrows():
  player_id = player['id']
  player_name = f"{player['first_name']}_{player['second_name']}".replace(' ', '_')
  player = requests.get(f'https://fantasy.allsvenskan.se/api/element-summary/{player_id}/')
  data = player.json()
  player_history = pd.DataFrame(data['history'])
  player_history.to_csv(f'data/player_history/{player_id}_{player_name}.csv', index=False)

## Merge all players game history to a single DataFrame

In [6]:
player_history_dir = os.fsencode('data/player_history')

players_history = pd.DataFrame()
for file in os.listdir(player_history_dir):
  filename = f'data/player_history/{os.fsdecode(file)}'
  
  try:
    file_data = pd.read_csv(filename, index_col=False, header=0)
  except pd.errors.EmptyDataError:
    file_data = pd.DataFrame()
  players_history = players_history.append(file_data)

players_history.sort_values(by=['element', 'round'], inplace=True)
players_history.to_csv('data/player_game_history.csv', index=False)

players_history.sample(5)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,own_goals,attacking_bonus,defending_bonus,crosses,key_passes,big_chances_created,clearances_blocks_interceptions,recoveries,value,transfers_balance,selected,transfers_in,transfers_out
13,213,112,14,0,True,2021-08-07T15:30:00Z,2,2,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,-1,32,0,1
7,125,103,1,0,False,2021-05-20T16:30:00Z,2,1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,78,-293,1356,1,294
0,346,6,14,0,False,2021-04-11T13:00:00Z,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,589,0,0
8,197,67,7,0,False,2021-07-05T17:00:00Z,3,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48,-7,333,0,7
0,6,1,2,8,True,2021-04-10T13:00:00Z,3,2,1,90,1,0,0,2,0,0,0,0,0,0,0,1,2,2,0,3,6,70,0,3044,0,0


## Make a clean dataset of the interesting columns

In [7]:
player_game_history_clean = players_history[["element", "round", "opponent_team", "was_home", "total_points"]]
player_game_history_clean.sample(10)

,element,round,opponent_team,was_home,total_points
7,323,8,6,False,4
4,140,5,4,False,0
4,242,5,12,True,0
3,387,4,4,True,0
9,292,10,4,True,0
11,333,12,4,True,4
9,305,10,13,True,8
9,58,10,1,False,0
8,399,9,11,True,2
15,256,16,11,False,0


## Set player id, player name and team names

In [8]:
# player_id
player_game_history_clean = player_game_history_clean.rename(columns={'element': 'player_id', 'opponent_team': 'opponent_team_id'})

# player_name
player_data["player_name"] = player_data["first_name"] + ' ' + player_data["second_name"]
player_data["player_name"]
player_game_history_clean["player_name"] = player_game_history_clean["player_id"].map(player_data.set_index("id")["player_name"])

# team_name
player_game_history_clean["team_id"] = player_game_history_clean["player_id"].map(player_data.set_index("id")["team"])
player_game_history_clean["team_name"] = player_game_history_clean["team_id"].map(teams_data.set_index("id")["name"])

# opp_team_name
player_game_history_clean["opponent_team_name"] = player_game_history_clean["opponent_team_id"].map(teams_data.set_index("id")["name"])

# Reorder columns
player_game_history_clean_final = player_game_history_clean[["round", "player_name", "team_name", "opponent_team_name", "was_home", "total_points"]]
player_game_history_clean_final

,round,player_name,team_name,opponent_team_name,was_home,total_points
0,1,Johan Dahlin,Malmö FF,Hammarby,True,0
1,2,Johan Dahlin,Malmö FF,BK Häcken,False,0
2,3,Johan Dahlin,Malmö FF,Östersunds FK,True,0
3,4,Johan Dahlin,Malmö FF,Djurgården,False,0
4,5,Johan Dahlin,Malmö FF,Varbergs BoIS,True,0
...,...,...,...,...,...,...
1,16,Karl Serrano,Degerfors IF,Malmö FF,False,0
0,16,Nikola Djurdjic,Degerfors IF,Malmö FF,False,0
0,16,Ahmed Yasin Ghani Mousa,Örebro SK,IK Sirius,True,0
0,16,Rasmus Bonde,AIK,BK Häcken,True,0


In [9]:
player_game_history_clean_final.to_csv('data/player_game_history_clean.csv', index=False)